In [168]:
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn import linear_model
from sklearn import preprocessing
from scipy.stats import skew
from scipy.stats.stats import pearsonr

%matplotlib inline

In [153]:
fields = ['host_response_rate', 'host_is_superhost', 'smart_location', 'property_type', 'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type', 'square_feet', 'guests_included', 'minimum_nights', 'maximum_nights', 'number_of_reviews','review_scores_rating','review_scores_accuracy','review_scores_cleanliness','review_scores_checkin','review_scores_communication','review_scores_location','review_scores_value','instant_bookable','cancellation_policy','calculated_host_listings_count','reviews_per_month','price']

data = pd.read_csv('./airbnb_nyc_listings.csv', usecols = fields)
data = data.apply(lambda x: x.fillna(0) if x.dtype.kind in 'iufc' else x.fillna('NA'))
train = data.sample(frac = 0.80, random_state = 200)
test = data.drop(train.index)
test.head()

,host_response_rate,host_is_superhost,smart_location,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,calculated_host_listings_count,reviews_per_month
2,100%,f,"Queens, NY",Apartment,Private room,2,1.0,1.0,1.0,Real Bed,...,10.0,9.0,10.0,10.0,9.0,9.0,f,flexible,1,1.81
3,NaN,f,"New York, NY",House,Shared room,2,1.0,1.0,1.0,Real Bed,...,NaN,NaN,NaN,NaN,NaN,NaN,f,moderate,1,NaN
4,100%,f,"New York, NY",Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,...,10.0,10.0,10.0,10.0,10.0,10.0,f,flexible,1,1.00
5,NaN,f,"Queens, NY",Apartment,Private room,1,1.0,1.0,1.0,Real Bed,...,NaN,NaN,NaN,NaN,NaN,NaN,f,moderate,1,NaN
17,NaN,f,"Brooklyn, NY",Apartment,Entire home/apt,4,1.0,1.0,2.0,Real Bed,...,NaN,NaN,NaN,NaN,NaN,NaN,f,flexible,1,NaN


In [154]:
# we don't need the price right now
indexOfPrice = train.columns.get_loc('price')
all_data = pd.concat((train.loc[:,'host_response_rate':'reviews_per_month'],
                      test.loc[:,'host_response_rate':'reviews_per_month']))

In [162]:
# make the price column numeric
train['price'] = train['price'].replace('[\$,]', '', regex=True).astype(float)
test['price'] = test['price'].replace('[\$,]', '', regex=True).astype(float)

In [179]:
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
prices = pd.DataFrame({"price":train["price"], "log(price + 1)":np.log1p(train["price"])})
#prices.hist()

In [183]:
train['square_feet'].isnull().sum()/len(train['square_feet'])

0.98492946367534651